In [8]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import pandas as pd

import config

In [9]:
# Build function to load dataset into azure database
HOST = config.settings["host"]
MASTER_KEY = config.settings["master_key"]
DATABASE_ID = config.settings["database_id"]
CONTAINER_ID = config.settings["container_id"]

def push(jsonbody):
    client = cosmos_client.CosmosClient(
        HOST,
        {"masterKey": MASTER_KEY},
        user_agent="CosmosDBPythonQuickstart",
        user_agent_overwrite=True,
    )
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print("Database with id '{0}' created".format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print("Database with id '{0}' was found".format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(
                id=CONTAINER_ID, partition_key=PartitionKey(path="/partitionKey")
            )
            print("Container with id '{0}' created".format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print("Container with id '{0}' was found".format(CONTAINER_ID))

        container.create_item(body=jsonbody)

    except exceptions.CosmosHttpResponseError as e:
        print("\nrun_sample has caught an error. {0}".format(e.message))

    finally:
        print("\nrun_sample done")

AttributeError: module 'config' has no attribute 'settings'

In [ ]:
# Read dataset
df = pd.read_csv('github_repos_dataset.csv')
df

,Unnamed: 0,url,repo,author,license,watch_count,languages
0,0,https://github.com/01miru/HomeSense,HomeSense,01miru,mit,3,"['Swift', 'Other']"
1,1,https://github.com/01org/yask,yask,01org,mit,6,"['C++', 'Perl', 'Makefile', 'Python', 'Other']"
2,2,https://github.com/06wj/FL,FL,06wj,mit,3,"['JavaScript', 'Python']"
3,4,https://github.com/0of/Promise2,Promise2,0of,mit,4,"['C++', 'CMake', 'Other']"
4,5,https://github.com/0racle/p6-wig,p6-wig,0racle,artistic-2.0,3,['Other']
5,6,https://github.com/0rax/slack-flip-bot,slack-flip-bot,0rax,mit,4,['Go']
6,7,https://github.com/0x00A/cpp-eventemitter,cpp-eventemitter,0x00A,mit,8,['C++']
7,8,https://github.com/0x00A/nodeuv-http,nodeuv-http,0x00A,mit,9,"['C++', 'Python', 'Makefile', 'Batchfile']"
8,9,https://github.com/0x09/dspfun,dspfun,0x09,mit,7,"['C', 'Makefile']"


In [ ]:
# Iterate on each row
i = 0
for index, url, repo, author, license, watch_count, languages in df.itertuples():
    jsonbody = {
            "id": index,
            "repository": repo,
            "watchers": watch_count,
            "license": license,
            "language": languages,
            "topic": "",
        }
    push(jsonbody)